# ISO-SWS data preprocessing: convert to pickled dataframes

In [2]:
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.io import fits
from astropy.table import Table
from IPython.core.debugger import set_trace as st
from scipy.interpolate import splev, splrep

In [3]:
# Some useful functions....
cassis_wave = np.loadtxt('isosws_misc/cassis_wavelength_grid.txt', delimiter=',')

def convert_fits_to_pickle(path, verify_pickle=False, verbose=False, match_cassis_wavegrid=False):
    """Full conversion from ISO-SWS <filename.fits to <filename>.pkl, which contains a pd.DataFrame.
    
    Args:
        path (str): Path to <filename>.fits file (of an ISO-SWS observation).
        verify_pickle (bool): Confirm the pickle was succesful created; does so by comparing the
            pd.DataFrame before and after writing the pickle.
        
    Returns:
        True if successful.
        
    Note:
        DataFrame can be retrieved from the pickle by, e.g., df = pd.read_pickle(pickle_path).
    """
    
    if verbose:
        print('Pickling: ', path)
    
    # Convert .fits file to pandas DataFrame, header.Header object.
    df, header = isosws_fits_to_dataframe(path)

    # Downsample to match the CASSIS wavegrid if desired.
    if match_cassis_wavegrid:
        df = downsample_to_cassis(df)
    
    # Determine the pickle_path to save to. Being explicit here to 'pickle_path' is clear.
    base_filename = path.replace('.fit', '.pkl').split('/')[-1]
    
    # Save the dataframe to a pickle.
    pickle_path = 'isosws_dataframes/' + base_filename
    df.to_pickle(pickle_path)
    
    if verbose:
        print('...saved: ', pickle_path)

    # Test dataframes for equality before/after pickling if verify_pickle == True.
    if verify_pickle:
        tmp_df = pd.read_pickle(pickle_path)
        if df.equals(tmp_df):
            if verbose:
                print()
                print('DataFrame integrity verified -- pickling went OK!')
                print()
        else:
            raise ValueError('Dataframes not equal before/after pickling!')
    
    return pickle_path


def isosws_fits_to_dataframe(path, test_for_monotonicity=True):
    """Take an ISO-SWS .fits file, return a pandas DataFrame containing the data (with labels) and astropy header.
    
    Args:
        path (str): Path of the .fits file (assumed to be an ISO-SWS observation file).
        test_for_monotonicity (bool, optional): Check that the wavelength grid is monotinically increasing.
        
    Returns:
        df (pd.DataFrame): Pandas dataframe with appropriate labels (wavelength, flux, etc.).
        header (astropy.io.fits.header.Header): Information about observation from telescope.
        
    Note:
        Header can be manipulated with, e.g., header.totextfile(some_path).
        See http://docs.astropy.org/en/stable/io/fits/api/headers.html.
    """
    
    def monotonically_increasing(array):
        """Test if a list has monotonically increasing elements. Thank you stack overflow."""
        return all(x < y for x, y in zip(array, array[1:]))
    
    # Read in .fits file.
    hdu = fits.open(path)
    
    # Retrieve the header object.
    header = hdu[0].header
    
    # Extract column labels/descriptions from header.
    # Can't do this because the header is not well-defined. That's OK, hard-coded the new column names below.
    
    # Convert data to pandas DataFrame.
    dtable = Table(test_hdu[0].data)
    df = dtable.to_pandas()
    
    # Convert the nondescriptive column labels (e.g., 'col01def', 'col02def') to descriptive labels.
    old_keys = list(df.keys())
    new_keys = ['wavelength', 'flux', 'spec_error', 'norm_error']
    mydict = dict(zip(old_keys, new_keys))
    df = df.rename(columns=mydict)  # Renamed DataFrame columns here.
    
    if test_for_monotonicity:
         if not monotonically_increasing(df['wavelength']):
                raise ValueError('Wavelength array not monotonically increasing!', path)
    
    return df, header


def downsample_to_cassis(df):
    """Downsample to match the wavelength grid of CASSIS."""
    
    wave = df['wavelength']
    flux = df['flux']
    spec_error = df['spec_error']
    norm_error = df['norm_error']

    def spline(x, y, new_x):
        spline_model = splrep(x=x, y=y)
        new_y = splev(x=new_x, tck=spline_model)
        return new_y

    new_wave = cassis_wave
    new_flux = spline(wave, flux, new_wave)
    new_spec_error = spline(wave, spec_error, new_wave)
    new_norm_error = spline(wave, norm_error, new_wave)

    col_stack = np.column_stack([new_wave, new_flux, new_spec_error, new_norm_error])
    col_names = ['wavelength', 'flux', 'spec_error', 'norm_error']

    df2 = pd.DataFrame(col_stack, columns=col_names)
    
    return df2


***

## Find out how many files we're working with

In [4]:
spec_dir = 'spectra/'
spec_files = np.sort(glob.glob(spec_dir + '*.fit'))

In [5]:
len(spec_files)

1262

## Convert spectra to dataframes and save to disk as pickles

In [8]:
perform_conversion = False

In [9]:
# Note the break I've added; remove for full conversion.
if perform_conversion:
    print('=============================\nConverting fits files...\n=============================\n')

    # Iterate over all the fits files and convert them.
    for index, fits_file in enumerate(spec_files):
#         if index >= 22:
#             break

        if index % 20 == 0:
            print(index, '/', len(spec_files))

        pickle_path = convert_fits_to_pickle(fits_file, verify_pickle=True, verbose=False, match_cassis_wavegrid=True)

    print('\n=============================\nComplete.\n=============================')

## Build dataframe containing metadata (including labels) and paths to pickled files.

###### Creates isosws_metadata_df.pkl.

In [10]:
# Only do this once.
recreate_meta_pickle = True

if recreate_meta_pickle:
    def create_swsmeta_dataframe():
        """Create a dataframe that contains the metadata for the ISO-SWS Atlas."""
        
        def simbad_results():
            """Create a dictionary of the SIMBAD object type query results."""
            simbad_results = np.loadtxt('isosws_misc/simbad_type.csv', delimiter=';', dtype=str)
            simbad_dict = dict(simbad_results)
            return simbad_dict
        
        def sexagesimal_to_degree(tupe):
            """Convert from hour:minute:second to degrees."""
            sex_str = tupe[0] + ' ' + tupe[1]
            c = SkyCoord(sex_str, unit=(u.hourangle, u.deg))
            return c.ra.deg, c.dec.deg        
        
        def transform_ra_dec_into_degrees(df):
            """Perform full ra, dec conversion to degrees."""
            ra = []
            dec = []
            for index, value in enumerate(zip(df['ra'], df['dec'])):
                ra_deg, dec_deg = sexagesimal_to_degree(value)
                ra.append(ra_deg)
                dec.append(dec_deg)
            df = df.assign(ra=ra)
            df = df.assign(dec=dec)
            return df

        # Read in the metadata
        meta_filename = 'isosws_misc/kraemer_class.csv'
        swsmeta = np.loadtxt(meta_filename, delimiter=';', dtype=str)
        df = pd.DataFrame(swsmeta[1:], columns=swsmeta[0])
        
        # Add a column for the pickle paths (dataframes with wave, flux, etc).
        pickle_paths = ['isosws_dataframes/' + x + '_sws.pkl' for x in df['tdt']]
        df = df.assign(file_path=pickle_paths)
        
        # Add a column for SIMBAD type, need to query 'simbad_type.csv' for this. Not in order naturally...
        object_names = df['object_name']
        object_type_dict = simbad_results()
        object_types = [object_type_dict.get(key, "empty") for key in object_names]
        df = df.assign(object_type=object_types)

        # Transform ra and dec into degrees.
        df = transform_ra_dec_into_degrees(df)
        
        return df
    
    df = create_swsmeta_dataframe()
    df.to_pickle('isosws_metadata_df.pkl')

In [11]:
df.head()

,object_name,tdt,ra,dec,full_classifier,group,subgroup,uncertainty_flag,note,,file_path,object_type
0,W Cet,37802225,0.532083,-14.676639,2.SEa:,2,SEa,1,,,isosws_dataframes/37802225_sws.pkl,S*
1,SV And,42801007,1.083333,40.110333,2.SEa:,2,SEa,1,,,isosws_dataframes/42801007_sws.pkl,Mira
2,SV And,80800708,1.083333,40.110333,2.SEa,2,SEa,,,,isosws_dataframes/80800708_sws.pkl,Mira
3,CIT 1,78201008,1.717917,43.076667,7,7,,,W,,isosws_dataframes/78201008_sws.pkl,Mira
4,HR 10,37802001,1.825833,-17.387000,1.NM:,1,NM,1,,,isosws_dataframes/37802001_sws.pkl,Star


In [12]:
mdf = pd.read_pickle('isosws_metadata_df.pkl')
mdf

,object_name,tdt,ra,dec,full_classifier,group,subgroup,uncertainty_flag,note,,file_path,object_type
0,W Cet,37802225,0.532083,-14.676639,2.SEa:,2,SEa,1,,,isosws_dataframes/37802225_sws.pkl,S*
1,SV And,42801007,1.083333,40.110333,2.SEa:,2,SEa,1,,,isosws_dataframes/42801007_sws.pkl,Mira
2,SV And,80800708,1.083333,40.110333,2.SEa,2,SEa,,,,isosws_dataframes/80800708_sws.pkl,Mira
3,CIT 1,78201008,1.717917,43.076667,7,7,,,W,,isosws_dataframes/78201008_sws.pkl,Mira
4,HR 10,37802001,1.825833,-17.387000,1.NM:,1,NM,1,,,isosws_dataframes/37802001_sws.pkl,Star
5,{beta} Cas,28501420,2.293625,59.149944,1.N,1,N,,,,isosws_dataframes/28501420_sws.pkl,Star
6,V633 Cas,43501514,2.860833,58.834444,5.SE,5,SE,,W,,isosws_dataframes/43501514_sws.pkl,Ae*
7,NGC 40,44401917,3.253792,72.522222,4.PN,4,PN,,,,isosws_dataframes/44401917_sws.pkl,PN
8,NGC 40,30003803,3.254583,72.521972,4.PN,4,PN,,,,isosws_dataframes/30003803_sws.pkl,PN
9,HR 48,55502138,3.660000,-18.932889,1.NO,1,NO,,,,isosws_dataframes/55502138_sws.pkl,Candidate_LP*


***

***

***

# Appendix A -- Example transformation from .fits to pd.dataframe

#### Convert spectrum file to dataframe, header

In [63]:
# Grab the first file from the glob list.
test_spec = spec_files[0]
test_spec

'spectra/02400714_sws.fit'

In [64]:
# Read it in with astropy.io.fits, check dimensions.
test_hdu = fits.open(test_spec)
test_hdu.info()

Filename: spectra/02400714_sws.fit
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      52   (4, 48924)   float32   


In [68]:
# Utilize our defined function to transform a string of the .fits filename to a pandas dataframe and header.
# 'header' will be an astropy.io.fits.header.Header object; see a couple subsections below for conversion options.
df, header = isosws_fits_to_dataframe(test_spec)

#### Inspect dataframe

In [69]:
df.shape

(48924, 4)

In [70]:
df.head()

,wavelength,flux,spec_error,norm_error
0,2.36000,-3.34,0.52,0.52
1,2.36013,-3.23,0.52,0.52
2,2.36025,-3.13,0.52,0.52
3,2.36038,-3.06,0.51,0.51
4,2.36050,-3.01,0.51,0.51


In [71]:
df.describe()

,wavelength,flux,spec_error,norm_error
count,48924.000000,48924.000000,48924.000000,48924.000000
mean,13.890253,29.865271,1.316019,1.436715
std,11.323857,77.273079,2.070261,2.219514
min,2.360000,-6.720000,0.080000,0.080000
25%,4.717688,-2.600000,0.270000,0.390000
50%,10.250250,-0.660000,0.560000,0.600000
75%,19.009378,43.250000,0.990000,1.000000
max,45.389999,3170.389893,16.940001,17.129999


#### Header from the .fits file

In [72]:
type(header)

astropy.io.fits.header.Header

In [73]:
# Uncomment below to see full header of one file as an example.
header

SIMPLE  =                    T / Written by IDL:  Wed Apr  9 10:12:29 2003      
BITPIX  =                  -32 / Number of bits per data pixel                  
NAXIS   =                    2 / Number of data axes                            
NAXIS1  =                    4 /                                                
NAXIS2  =                48924 /                                                
NSEG    =                   12 / Number of spectral segments                    
NSEG01  =                 1921 / Length of segment  1                           
NSEG02  =                 2520 / Length of segment  2                           
NSEG03  =                 3000 / Length of segment  3                           
NSEG04  =                 2240 / Length of segment  4                           
NSEG05  =                 4881 / Length of segment  5                           
NSEG06  =                 3400 / Length of segment  6                           
NSEG07  =                108

In [74]:
# Can convert to other formats if we want to use the header information for something.
# See http://docs.astropy.org/en/stable/io/fits/api/headers.html

# header_str = header.tostring()
# header.totextfile('test_header.csv')

***